# GNSS Dropout MTIE Plot

This notebook demonstrates a GNSS dropout scenario and plots MTIE for the fused clock.

Config used: `configs/gnss_dropout.yaml`.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/infinityabundance/ai-multi-reference-timekeeping/blob/main/notebooks/12_gnss_dropout_mtie.ipynb)


In [ ]:
import os
if not os.path.exists('ai-multi-reference-timekeeping'):
    !git clone https://github.com/infinityabundance/ai-multi-reference-timekeeping.git
%cd ai-multi-reference-timekeeping
!pip install -e .


In [ ]:
import matplotlib.pyplot as plt
from ai_multi_reference_timekeeping.fusion import HeuristicFusion, Measurement, VirtualClock
from ai_multi_reference_timekeeping.kalman import ClockCovariance, ClockKalmanFilter, ClockState
from ai_multi_reference_timekeeping.metrics import mtie

kalman = ClockKalmanFilter(
    state=ClockState(offset=0.0, drift=0.0),
    covariance=ClockCovariance(p00=1.0, p01=0.0, p10=0.0, p11=1.0),
    process_noise_offset=1e-4,
    process_noise_drift=1e-6,
)
clock = VirtualClock(kalman_filter=kalman, fusion=HeuristicFusion())

offsets = []
for step in range(600):
    measurements = [Measurement(name='ptp', offset=0.002, variance=1e-3)]
    if step < 180 or step > 420:
        measurements.append(Measurement(name='gnss', offset=0.0, variance=1e-4))
    update = clock.step(1.0, measurements)
    offsets.append(update.fused_offset)

window = 10
mtie_values = [mtie(offsets[: i + 1], window=window) if i + 1 >= window else 0 for i in range(len(offsets))]
plt.plot(mtie_values)
plt.title('MTIE over time (GNSS dropout scenario)')
plt.xlabel('Time (s)')
plt.ylabel('MTIE (s)')
plt.show()
